# Experiment 4 - following directly the algorithm in the paper

In [341]:
import os

import pandas as pd
from rdflib import Graph, URIRef

In [342]:
def get_nt_files(directory):
    nt_files = []
    for file in os.listdir(directory):
        if file.endswith(".nt"):
            nt_files.append(os.path.join(directory, file))
    return nt_files


def nt_files_to_dataframe(directory):
    nt_files = get_nt_files(directory)
    nt_files_df = pd.DataFrame(nt_files, columns=["nt_file"])
    return nt_files_df


directory = "data/lubm1_intact/graphs_with_descriptions"
nt_files_df = nt_files_to_dataframe(directory)
nt_files_df = nt_files_df.sort_values(by="nt_file").reset_index().drop(columns='index')

In [343]:
nt_files_df.head(10)

,nt_file
0,data/lubm1_intact/graphs_with_descriptions/HTT...
1,data/lubm1_intact/graphs_with_descriptions/HTT...
2,data/lubm1_intact/graphs_with_descriptions/HTT...
3,data/lubm1_intact/graphs_with_descriptions/HTT...
4,data/lubm1_intact/graphs_with_descriptions/HTT...
5,data/lubm1_intact/graphs_with_descriptions/HTT...
6,data/lubm1_intact/graphs_with_descriptions/HTT...
7,data/lubm1_intact/graphs_with_descriptions/HTT...
8,data/lubm1_intact/graphs_with_descriptions/HTT...
9,data/lubm1_intact/graphs_with_descriptions/HTT...


In [372]:
import ast


def read_csv_extract_columns(file_path, columns):
    data = pd.read_csv(file_path)
    extracted_data = data[columns]
    return extracted_data


csv_file_path = "code/encoding.csv"
columns = ["input_graph_file", "input_graph_encoding", "inference_file", "inference_graph_encoding"]
extracted_data = read_csv_extract_columns(csv_file_path, columns)
extracted_data['input_graph_file'] = extracted_data['input_graph_file'].str.replace('^../', '')
extracted_data['inference_graph_file'] = extracted_data['inference_graph_file'].str.replace('^../', '')
extracted_data.input_graph_encoding = extracted_data.input_graph_encoding.apply(lambda x: ast.literal_eval(x))
extracted_data.inference_graph_encoding = extracted_data.inference_graph_encoding.apply(lambda x: ast.literal_eval(x))

KeyError: "['inference_graph_file', 'inference_graph_encoding'] not in index"

In [345]:
extracted_data.head(10)

,input_graph_file,input_graph_encoding
0,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 1)], 2: [(-1, -2)], 3: [(-3, -2), (-..."
1,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 3)], 7: [(-1, -2), (-3, -2), (-4, -2..."
2,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2), (..."
3,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2)]}"
4,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2), (..."
5,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2), (..."
6,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2), (..."
7,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2)]}"
8,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 3)], 7: [(-1, -2), (-3, -2), (-4, -2..."
9,data/lubm1_intact/graphs_with_descriptions/HTT...,"{1: [(-1, 8)], 4: [(-1, -2)], 11: [(-1, -2), (..."


In [346]:
nt_files_df.nt_file[0]

'data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu.nt'

In [347]:
# Load ontology and RDF graphs
ontology_file_tbox = "data/updated/ub.nt"
ontology_file_abox = "data/lubm1_intact/all_lubm.nt"
inference_graph_file = "data/lubm1_intact/jena_inference_with_descriptions/HTTP_www.Department0.University0.edu.nt"

rdf_ontology = Graph()
rdf_ontology.parse(ontology_file_tbox, format="nt")
rdf_ontology.parse(ontology_file_abox, format="nt")
rdf_inference_graph = Graph()
rdf_inference_graph.parse(inference_graph_file, format="nt")

<Graph identifier=N943ba9b3749c40fab5409da8d05f63fc (<class 'rdflib.graph.Graph'>)>

In [348]:
def create_graphs_from_files(dataframe, file_column):
    graphs = []
    for file_path in dataframe[file_column]:
        g = Graph()
        full_path = os.path.abspath(file_path)
        g.parse(full_path, format='nt')
        graphs.append(g)
    return graphs


extracted_data['graph'] = create_graphs_from_files(extracted_data, 'input_graph_file')

In [349]:
extracted_data.graph[0]

<Graph identifier=Nb5dbe64f5aaa4ad8948be39686b38652 (<class 'rdflib.graph.Graph'>)>

In [350]:
# Step 1: Create properties dictionary
property_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?property WHERE {
  {
    ?subject rdf:type ?object .
    BIND(rdf:type as ?property)
  } UNION {
    ?property a owl:ObjectProperty .
  } UNION {
    ?property a owl:DatatypeProperty .
  } UNION {
    ?property a owl:TransitiveProperty .
  }
}
"""

In [351]:
# Define custom sorting function
def custom_sort(property_uri):
    property_str = str(property_uri)
    if property_str == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
        return ('', property_str)
    else:
        return (property_str,)


# Get the properties as a list and sort them
property_list = [row.property for row in rdf_ontology.query(property_query)]
sorted_properties = sorted(property_list, key=custom_sort)

# Generate the properties dictionary
properties_dictionary = {property: index + 1 for index, property in enumerate(sorted_properties)}

In [352]:
len(properties_dictionary)  ## SHOULD BE 32 - paper page 26 line 36

33

In [353]:
# class_query = """
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX owl: <http://www.w3.org/2002/07/owl#>
# SELECT DISTINCT ?class_name WHERE {
#   {
#     ?class_name a rdfs:Class .
#   } UNION {
#     ?class_name a owl:Class .
#   }
#   FILTER(isURI(?class_name))
# }
# """
#
# unique_class_names = set(row.class_name for row in rdf_ontology.query(class_query))
# sorted_class_names = sorted(unique_class_names, key=lambda x: str(x))
#
# global_resources_dictionary = {class_name: index + 1 for index, class_name in enumerate(sorted_class_names)}

In [354]:
class_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?class_name WHERE {
  {
    ?class_name a rdfs:Class .
  } UNION {
    ?class_name a owl:Class .
  }
  FILTER(isURI(?class_name))
}
"""

global_resources_dictionary = {}
for i, row in enumerate(rdf_ontology.query(class_query)):
    global_resources_dictionary[row.class_name] = i

In [355]:
len(global_resources_dictionary)  #SHOULD BE 57 - paper page 26 line 45

43

In [356]:
global_resources_dictionary

{rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#AdministrativeStaff'): 0,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Article'): 1,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#AssistantProfessor'): 2,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#AssociateProfessor'): 3,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Book'): 4,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Chair'): 5,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Person'): 6,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Department'): 7,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#ClericalStaff'): 8,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#College'): 9,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#ConferencePaper'): 10,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto

# Adding Missing Base Classes

In [357]:
import rdflib

base_classes = [
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#Alt",
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#Bag",
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#List",
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#Property",
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#Seq",
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement",
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral",
    "http://www.w3.org/2000/01/rdf-schema#Class",
    "http://www.w3.org/2000/01/rdf-schema#Container",
    "http://www.w3.org/2000/01/rdf-schema#ContainerMembershipProperty",
    "http://www.w3.org/2000/01/rdf-schema#Datatype",
    "http://www.w3.org/2000/01/rdf-schema#Literal",
    "http://www.w3.org/2000/01/rdf-schema#Resource",
    "http://www.w3.org/2001/XMLSchema#nonNegativeInteger",
    "http://www.w3.org/2001/XMLSchema#string"
]

for base_class in base_classes:
    if rdflib.URIRef(base_class) not in global_resources_dictionary:
        global_resources_dictionary[rdflib.URIRef(base_class)] = len(global_resources_dictionary)

In [358]:
len(global_resources_dictionary)  #SHOULD BE 57 - paper page 26 line 45

58

In [359]:
global_resources_dictionary

{rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#AdministrativeStaff'): 0,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Article'): 1,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#AssistantProfessor'): 2,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#AssociateProfessor'): 3,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Book'): 4,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Chair'): 5,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Person'): 6,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#Department'): 7,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#ClericalStaff'): 8,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#College'): 9,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto/univ-bench.owl#ConferencePaper'): 10,
 rdflib.term.URIRef('http://swat.cse.lehigh.edu/onto

# I don't know WHICH ONE is wrong because all 58 of them are in his final file

# GLOBAL PROPERTIES AND RESOURCES DICTIONARIES DONE

In [360]:
import networkx as nx
from rdflib.plugins.sparql import prepareQuery

subproperty_query = prepareQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?property1 ?property2
    WHERE {
        ?property1 rdfs:subPropertyOf ?property2
        FILTER(?property1 != ?property2)
    }
""")

subproperty_graph = nx.Graph()

for row in rdf_ontology.query(subproperty_query):
    subproperty_graph.add_edge(row.property1, row.property2)

In [361]:
subgraphs = list(nx.connected_components(subproperty_graph))

In [362]:
properties_groups = {}

for group_id, subgraph in enumerate(subgraphs):
    for property in subgraph:
        properties_groups[property] = group_id


In [363]:
# Find the maximum group_id used so far
max_group_id = max(properties_groups.values()) if properties_groups else -1

# Add the properties from properties_dictionary to the properties_groups
for property in properties_dictionary:
    if property not in properties_groups:
        max_group_id += 1
        properties_groups[property] = max_group_id

# PROPERTIES GROUPS DONE - START ENCODING

In [364]:
# GLOBAL PROPERTIES DICTIONARY - properties_dictionary - already exists
global_properties_dictionary = properties_dictionary
global_active_properties_dictionary = {}
# GLOBAL RESOURCES DICTIONARY - global_resources_dictionary - already exists
global_active_resources_dictionary = {}
# GLOBAL PROPERTY GROUPS DICTIONARY  - properties_groups
global_property_groups_dictionary = properties_groups  # already exists
# LOCAL RESOURCES DICTIONARY [graph name] -> (local props, local resources)
# local_resources_dictionaries = {}
global_local_resources_dictionary = {}
global_property_IDs = {}

In [365]:
def add_resource(resource, property, local_resources_dictionary):
    property_group = global_property_groups_dictionary[property]

    if resource in global_active_resources_dictionary:
        return

    if property not in global_active_properties_dictionary:
        global_active_properties_dictionary[property] = len(global_active_properties_dictionary) + 1

    if resource in global_resources_dictionary and resource not in global_active_resources_dictionary:
        global_active_resources_dictionary[resource] = len(global_active_resources_dictionary) + 1

    else:
        if property_group not in local_resources_dictionary:
            local_resources_dictionary[property_group] = {}

        if resource not in local_resources_dictionary[property_group]:
            local_resources_dictionary[property_group][resource] = -(
                    len(local_resources_dictionary[property_group]) + 1)

In [366]:
def lookup_resource(resource, property, local_resources_dictionary):
    property_group = global_property_groups_dictionary[property]

    if property_group in global_active_resources_dictionary:
        return global_active_resources_dictionary[resource]
    else:
        if resource in global_active_resources_dictionary:
            return global_active_resources_dictionary[resource]
        else:
            return local_resources_dictionary[property_group][resource]

In [367]:
from rdflib import RDF, RDFS


def encode_advanced(rdf_graph, inference):
    if not inference:
        local_resources_dictionary = {}
    else:
        local_resources_dictionary = global_local_resources_dictionary

    unique_properties = {p for s, p, o in rdf_graph}
    filtered_property_list = [property for property in global_properties_dictionary if
                              URIRef(property) in unique_properties]

    for index, property in enumerate(filtered_property_list):
        if URIRef(property) not in global_property_IDs:
            global_property_IDs[URIRef(property)] = len(global_property_IDs) + 1

    sorted_triples = sorted(rdf_graph, key=lambda x: (global_property_IDs.get(x[1], float('inf')), x[0], x[2]))

    sparse_encoding = []
    encoding = {}

    for s, p, o in sorted_triples:
        if p not in properties_dictionary:
            continue

        if type(s) == rdflib.term.BNode or type(o) == rdflib.term.BNode:
            continue

        if (p, o) == (RDF.type, RDFS.Resource):
            continue

        if (p, o) == (RDF.type, RDFS.Class):
            continue

        if inference and o in global_resources_dictionary:
            global_active_resources_dictionary[o] = len(global_active_properties_dictionary) + 1

        if not inference:
            add_resource(s, p, local_resources_dictionary)
            add_resource(o, p, local_resources_dictionary)

        s_id = lookup_resource(s, p, local_resources_dictionary)
        o_id = lookup_resource(o, p, local_resources_dictionary)
        p_id = global_property_IDs[p]
        sparse_encoding.append((p_id, s_id, o_id))
        if p_id not in encoding:
            encoding[p_id] = []
        encoding[p_id].append((s_id, o_id))
        # print(f"{s}:{s_id}, {p}:{p_id}, {o}:{o_id}")

    return encoding


# ENCODING DONE - START TESTING

In [368]:
encodings = []

graphs_test = []
number_of_tests = 5

for i in range(number_of_tests):
    graphs_test.append(extracted_data.graph[i])

for graph in graphs_test:
    encodings.append(encode_advanced(graph, inference=False))

for z in range(number_of_tests):
    encoding1 = encodings[z]
    encoding2 = extracted_data.input_graph_encoding[z]


    def compare_dictionaries(dict1, dict2):
        mismatched_pairs = []
        for key in set(dict1.keys()).union(dict2.keys()):
            if key in dict1 and key in dict2:
                if dict1[key] != dict2[key]:
                    mismatched_pairs.append((key, dict1[key], dict2[key]))
            else:
                missing_in = "dict1" if key not in dict1 else "dict2"
                print(f"Key {key} is missing in {missing_in}")
        return mismatched_pairs


    mismatched_pairs = compare_dictionaries(encoding1, encoding2)

    print(f"Mismatched key-value pairs for layer {z}:")
    for key, value1, value2 in mismatched_pairs:
        print(f"Key {key}:")
        print("  Incorrect value:", value1)
        print("  Correct value:", value2)
    print()


Mismatched key-value pairs for layer 0:

Mismatched key-value pairs for layer 1:
Key 1:
  Incorrect value: [(-1, 2)]
  Correct value: [(-1, 3)]

Mismatched key-value pairs for layer 2:
Key 1:
  Incorrect value: [(-1, 3)]
  Correct value: [(-1, 8)]

Mismatched key-value pairs for layer 3:
Key 1:
  Incorrect value: [(-1, 3)]
  Correct value: [(-1, 8)]

Mismatched key-value pairs for layer 4:
Key 1:
  Incorrect value: [(-1, 3)]
  Correct value: [(-1, 8)]

